# Introducción
Para entrenar correctamente al modelo de *Machine Learning* es necesario recordar el flujo de trabajo dictado en el documento principal. La  

En una primera instancia, debemos cargar nuestro set de datos y eliminar las filas repetidas, esto se debe realizar para evitar que el modelo memorice los datos y pueda realizar buenas predicciones. Para manejar el set de datos, utilizaremos la librería Pandas. 

Una vez cargado el set de datos, hay que hacer un breve analisis de cada columna para determinar su tratamiento en particular. En un vistazo rápido, podemos observar que desde la columnas 4 (C#) hasta la columna 17 (NoSQL) estan compuestas por datos numericos y toman valores cero (0) o uno (1); estas 13 columnas no van a necesitar ser procesadas. Por otra parte, la columna "Experiencia" tambien está compuesta por valores numericos enteros, pero estos valores puden variar desde 0 hasta 20. La diferencia entre estos valores es muy grande, y si le introducimos estos valores al modelo, sin procesarlos anteriormente, le 

### Importamos la librería Pandas para manejar el dataset

In [68]:
import pandas as pd

### Cargamos el dataset y eliminamos las filas repetidas

In [69]:
data = pd.read_csv("candidatos.csv")
data.drop_duplicates(inplace=True)

### Importamos las herramientas necesarias para procesar las columnas categóricas

#### MinMaxScaler para la variable "Experiencia",  OrdinalEncoder para la variable "Educación" y "Aptitud",  OneHotEncoder para la variable "Área".

In [70]:
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, OneHotEncoder

In [71]:
educacion = ["Ninguna","Tecnicatura","Licenciatura","Ingeniería"]
aptitud = ["No apto", "Apto"]

In [72]:
mms = MinMaxScaler()
edu_encoder = OrdinalEncoder(categories=[educacion])
apt_encoder = OrdinalEncoder(categories=[aptitud])
ohe = OneHotEncoder(sparse_output=False).set_output(transform="pandas")

data[["Experiencia"]] = mms.fit_transform(data[["Experiencia"]])
data[["Aptitud"]] = apt_encoder.fit_transform(data[["Aptitud"]])
data[["Educación"]] = edu_encoder.fit_transform(data[["Educación"]])

area_encoded = ohe.fit_transform(data[["Área"]])
data = pd.concat([data, area_encoded], axis=1).drop(columns=["Área"])
data.rename(columns={
    "Área_Desarrollo Móvil": "Desarrollo Movil",
    "Área_Desarrollo Web": "Desarrollo Web",
    "Área_Desarrollo de Juegos": "Desarrollo de Juegos",
    },
    inplace=True
)

### En la etapa de entrenamiento

In [73]:
x = data.drop(columns=["Puntos","Aptitud"])
y = data["Aptitud"]

In [74]:
from sklearn.model_selection import train_test_split

In [75]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [76]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [77]:
models = {
    "LogisticRegression": LogisticRegression(),
    "DecisionTreeClassifier": DecisionTreeClassifier(),
}

accuracy_scores = {}

In [78]:
from sklearn.metrics import accuracy_score
from joblib import dump

In [79]:
for model_name, model in models.items():
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores[model_name] = accuracy
    print(f'Accuracy of {model_name} on test data: {accuracy}')
    print(f"Exactitud promedio entrenamiento: {model.score(x_train,y_train)}")
    print(f"Exactitud promedio validación: {model.score(x_test, y_test)}")
    print("Datos reales:\n", y_test)
    print("Datos predichos:\n", y_pred)
    with open(f"../models/{model_name}.joblib", "wb") as f:
        dump(model, f)

Accuracy of LogisticRegression on test data: 0.9144654088050315
Exactitud promedio entrenamiento: 0.9212725802102992
Exactitud promedio validación: 0.9144654088050315
Datos reales:
 23847    0.0
12063    0.0
842      1.0
330      0.0
9422     1.0
        ... 
25233    1.0
13333    1.0
24840    0.0
25226    1.0
3118     0.0
Name: Aptitud, Length: 1590, dtype: float64
Datos predichos:
 [0. 0. 1. ... 0. 1. 0.]
Accuracy of DecisionTreeClassifier on test data: 0.9572327044025157
Exactitud promedio entrenamiento: 1.0
Exactitud promedio validación: 0.9572327044025157
Datos reales:
 23847    0.0
12063    0.0
842      1.0
330      0.0
9422     1.0
        ... 
25233    1.0
13333    1.0
24840    0.0
25226    1.0
3118     0.0
Name: Aptitud, Length: 1590, dtype: float64
Datos predichos:
 [0. 0. 1. ... 0. 1. 0.]
